In [ ]:
import cv2
import os
import sqlite3
from datetime import datetime
from mtcnn import MTCNN  # For face detection
import numpy as np

detector = MTCNN()

if not os.path.exists('dataset'):
    os.makedirs('dataset')

conn = sqlite3.connect('face_detection.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS detections
             (name TEXT, detection_time TEXT)''')
conn.commit()

def capture_photos(name):
    cap = cv2.VideoCapture(0)
    count = 0
    capturing = False

    print(f"Press 's' to start capturing photos for {name}.")
    print("Press 'e' to stop capturing.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow('Capture Photos', frame)

        key = cv2.waitKey(1) & 0xFF

        if key == ord('s'):
            capturing = True
            print("Capturing started...")
        elif key == ord('e'):
            capturing = False
            print("Capturing stopped.")
            break

        if capturing:
            faces = detector.detect_faces(frame)
            for face in faces:
                x, y, width, height = face['box']
                face_img = frame[y:y + height, x:x + width]

                person_dir = f'dataset/{name}'
                if not os.path.exists(person_dir):
                    os.makedirs(person_dir)
                cv2.imwrite(f'{person_dir}/{count}.jpg', face_img)
                count += 1

    cap.release()
    cv2.destroyAllWindows()

def detect_faces():
    cap = cv2.VideoCapture(0)
    face_recognizer = cv2.face.LBPHFaceRecognizer_create()
    face_recognizer.read('face_model.yml')
    label_to_name = {i: name for i, name in enumerate(os.listdir('dataset'))}

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        faces = detector.detect_faces(frame)
        for face in faces:
            x, y, width, height = face['box']
            face_img = frame[y:y + height, x:x + width]
            gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)

            label, confidence = face_recognizer.predict(gray)
            print(f"Predicted Label: {label}, Confidence: {confidence}")  # Debug output

            if confidence < 70:
                name = label_to_name.get(label, "Unknown")
                detection_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

                c.execute("INSERT INTO detections (name, detection_time) VALUES (?, ?)",
                          (name, detection_time))
                conn.commit()

                print(f"Detected: {name} at {detection_time} with confidence {confidence}")

                cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
                cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        cv2.imshow('Face Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    name = input("Enter your name: ")

    capture_photos(name)

    faces, labels = [], []
    label_to_name = {}

    for i, person in enumerate(os.listdir('dataset')):
        label_to_name[i] = person
        for img_name in os.listdir(f'dataset/{person}'):
            img_path = f'dataset/{person}/{img_name}'
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                faces.append(img)
                labels.append(i)

    if len(faces) > 0 and len(labels) > 0:
        face_recognizer = cv2.face.LBPHFaceRecognizer_create()
        face_recognizer.train(faces, np.array(labels))
        face_recognizer.save('face_model.yml')
    else:
        print("No valid images found for training.")

    detect_faces()

    conn.close()